A great suggestion from a reviewer was to provide a summary description of the types of surfaces, number of surfaces vs. number of sites, and the chemical and structural variety of data. This notebook documents how we found these figures.

To make things simple, let's look at the entire dataset as a whole (without splits).

# Data gathering/parsing

In [1]:
import pickle


with open('../pull_data/gaspy/docs.pkl', 'rb') as file_handle:
    docs = pickle.load(file_handle)

Our `docs` is a list of dictionaries from Mongo (thus the name "doc" for "Mongo document", which is just a json/dictionary). Each json contains information needed to re-create an `ase.Atoms` object. Let's just take those verbose keys out and replace them with actual `Atoms` objects now that our `docs` are dictionaries, not just jsons.

And while we're at it, let's also add the bulk stoichiometry information.

In [2]:
from tqdm.notebook import tqdm
from gaspy.mongo import make_atoms_from_doc
from gaspy.atoms_operators import get_stoich_from_mpid


# Convert json information to atoms object
for doc in tqdm(docs, desc='Atoms'):
    atoms = make_atoms_from_doc(doc)
    doc['atoms'] = atoms
    del doc['results']
    del doc['calc']

    # Do it again for the initial configuration
    atoms_init = make_atoms_from_doc(doc['initial_configuration'])
    doc['initial_configuration'] = atoms_init

# Add stoichiometry info
mpids = {doc['mpid'] for doc in docs}
stoichs = {mpid: get_stoich_from_mpid(mpid) for mpid in tqdm(mpids, desc='Stoichiometries')}
for doc in docs:
    doc['stoich'] = stoichs[doc['mpid']]

# Show an example of a doc
docs[0]

{'mongo_id': ObjectId('5d83021130582ea2977b252c'),
 'adsorbate': 'H',
 'mpid': 'mp-1184026',
 'miller': [1, 1, 1],
 'shift': 0.0,
 'top': False,
 'coordination': 'Cu-Ru',
 'neighborcoord': ['Ru:Cu-Cu-Cu-Cu-Cu-Cu', 'Cu:Cu-Cu-Cu-Cu-Ru-Ru'],
 'energy': -0.49401431499998916,
 'atoms': Atoms(symbols='HCu18Ru6Cu18Ru6', pbc=True, cell=[[9.18163332, 0.0, 0.0], [-4.59081666, 4.97102071, 0.58578594], [0.0, 0.0, 33.95945733]], initial_charges=..., initial_magmoms=..., momenta=..., tags=..., constraint=FixAtoms(indices=[1, 3, 4, 5, 7, 11, 13, 16, 17, 19, 20, 22, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]), calculator=SinglePointCalculator(...)),
 'initial_configuration': Atoms(symbols='HCu18Ru6Cu18Ru6', pbc=True, cell=[[9.18163332, 0.0, 0.0], [-4.59081666, 4.97102071, 0.58578594], [0.0, 0.0, 33.95945733]], initial_charges=..., initial_magmoms=..., momenta=..., tags=..., constraint=FixAtoms(indices=[1, 3, 4, 5, 7, 11, 13, 16, 17, 19, 20, 22, 25, 

# Profiling
Thngs we could show:
- types of surfaces
- number of surfaces
- number of sites
- chemical variety
- structural variety

In [3]:
import pprint


millers = {tuple(doc['miller']) for doc in docs}
print('There are %i different Miller indices:\n%s' % (len(millers), pprint.pformat(millers)))

There are 42 different Miller indices:
{(0, 0, 1),
 (0, 1, -2),
 (0, 1, -1),
 (0, 1, 0),
 (0, 1, 1),
 (0, 1, 2),
 (0, 2, 1),
 (1, -2, -2),
 (1, -2, -1),
 (1, -2, 0),
 (1, -2, 1),
 (1, -1, -2),
 (1, -1, -1),
 (1, -1, 0),
 (1, -1, 1),
 (1, 0, -2),
 (1, 0, -1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 0, 2),
 (1, 1, -2),
 (1, 1, -1),
 (1, 1, 0),
 (1, 1, 1),
 (1, 1, 2),
 (1, 2, -2),
 (1, 2, -1),
 (1, 2, 0),
 (1, 2, 1),
 (1, 2, 2),
 (2, -1, 0),
 (2, -1, 1),
 (2, -1, 2),
 (2, 0, -1),
 (2, 0, 1),
 (2, 1, -2),
 (2, 1, -1),
 (2, 1, 0),
 (2, 1, 1),
 (2, 1, 2),
 (2, 2, -1),
 (2, 2, 1)}


In [4]:
from gaspy.gasdb import get_surface_from_doc


surfaces = {get_surface_from_doc(doc) for doc in docs}
print('There are %i different surfaces.' % len(surfaces))

successfully loaded your custom FW_config.yaml!
There are 9102 different surfaces.


In [5]:
from gaspy.gasdb import round_


def get_site_from_doc(doc):
    '''
    Modified version of `get_surface_from_doc`. Same idea, but more details.
    '''
    site = (doc['mpid'],
            tuple(doc['miller']),
            round_(doc['shift'], 2),
            doc['top'],
            tuple(doc['coordination'].split('-')),
            tuple(tuple(ncoord_str.split(':')[-1].split('-'))
                  for ncoord_str in doc['neighborcoord']))
    return site


sites = {get_site_from_doc(doc) for doc in docs}
print('There are %i different sites.' % len(sites))

There are 29843 different sites.


In [6]:
from collections import defaultdict
import pprint


# Count bulks
chemical_varieties = defaultdict(int)
for stoich in stoichs.values():
    n_elements = len(stoich)
    chemical_varieties[n_elements] += 1
for n_elements, n_bulks in sorted(chemical_varieties.items()):
    print('There are %s bulk structures with %s different components.' % (n_bulks, n_elements))

# Count calcs
ncalcs_by_nelements = defaultdict(int)
for doc in docs:
    n_elements = len(stoichs[doc['mpid']])
    ncalcs_by_nelements[n_elements] += 1

There are 61 bulk structures with 1 different components.
There are 1057 bulk structures with 2 different components.
There are 774 bulk structures with 3 different components.
There are 54 bulk structures with 4 different components.
There are 6 bulk structures with 5 different components.


In [7]:
from collections import defaultdict
import pprint
from pymatgen.ext.matproj import MPRester
from gaspy.utils import read_rc


# Get the spacegroup information from Materials Project
query = {'task_ids': {'$in': list(mpids)}}
required_info = ['task_ids', 'spacegroup']
with MPRester(read_rc('matproj_api_key')) as rester:
    results = rester.query(query, required_info)

# Parse the results and re-distribute to our docs
space_groups = {mpid: result['spacegroup']['crystal_system']
                for result in results
                for mpid in result['task_ids']}
structure_counts = defaultdict(int)
for doc in docs:
    crystal = space_groups[doc['mpid']]
    doc['crystal'] = crystal
    structure_counts[crystal] += 1

# Report
for crystal, count in structure_counts.items():
    print('We have %i different calculations with a %s structure.' % (count, crystal))

We have 6889 different calculations with a hexagonal structure.
We have 12339 different calculations with a orthorhombic structure.
We have 6155 different calculations with a monoclinic structure.
We have 9885 different calculations with a cubic structure.
We have 3592 different calculations with a trigonal structure.
We have 7670 different calculations with a tetragonal structure.
We have 749 different calculations with a triclinic structure.


In [8]:
from collections import defaultdict


counts_by_ads = defaultdict(int)
for doc in docs:
    counts_by_ads[doc['adsorbate']] += 1

print('There are %i different adsorbates:' % len(counts_by_ads))
for ads, count in counts_by_ads.items():
    print('%s encompasses %i of the calculations.' % (ads, count))

There are 5 different adsorbates:
H encompasses 21269 of the calculations.
CO encompasses 18437 of the calculations.
N encompasses 1594 of the calculations.
OH encompasses 3464 of the calculations.
O encompasses 2515 of the calculations.


Ok, so we got he basic information. Let's try and organize it a bit better.

In [9]:
from collections import defaultdict


def recursive_defaultdict():
    '''
    Credit goes to Andrew Clark on StackOverflow
    (https://stackoverflow.com/questions/19189274/nested-defaultdict-of-defaultdict)
    '''
    return defaultdict(recursive_defaultdict)


# Organize everything into a dictionary tree
dataset_description = recursive_defaultdict()
for doc in docs:
    adsorbate = doc['adsorbate']
    site = doc['coordination']
    surface = (tuple(doc['miller']), doc['shift'], doc['top'])
    bulk = doc['mpid']
    elements = tuple(stoichs[bulk].keys())
    crystal = space_groups[bulk]
    try:
        dataset_description[crystal][elements][bulk][surface][site][adsorbate] += 1
    except TypeError:
        dataset_description[crystal][elements][bulk][surface][site][adsorbate] = 1


def count_rec_dict(rec_dict):
    count = 0
    try:
        for subdict in rec_dict.values():
            count += count_rec_dict(subdict)
    except AttributeError:
        count += rec_dict
    return count
        

# Report
print('There are %i different crystals and %i calculations herein'
      % (len(dataset_description), count_rec_dict(dataset_description)))

for crystal, dict0 in dataset_description.items():
    print('  Within the %s crystal, there are %i different elemental combinations and %i calculations herein'
          % (crystal, len(dict0), count_rec_dict(dict0)))
    
    #for stoich, dict1 in dict0.items():
    #    print('    Within the %s combination, there are %i different stoichiometries and %i calculations herein'
    #          % (stoich, len(dict1), count_rec_dict(dict1)))
    #    
        #for bulk, dict2 in dict1.items():
        #    print('      Within the %s bulk structure, there are %i different surfaces and %i calculations herein'
        #          % (bulk, len(dict2), count_rec_dict(dict2)))
        #        
            #for surface, dict3 in dict2.items():
            #    print('        Within the %s surface, there are %i different sites and %i calculations herein'
            #          % (surface, len(dict3), count_rec_dict(dict3)))
            #    
                #for site, dict4 in dict3.items():
                #    print('          Within the %s sites, there are %i different adsorbates and %i calculations herein'
                #          % (site, len(dict4), count_rec_dict(dict4)))

There are 7 different crystals and 47279 calculations herein
  Within the hexagonal crystal, there are 215 different elemental combinations and 6889 calculations herein
  Within the orthorhombic crystal, there are 319 different elemental combinations and 12339 calculations herein
  Within the monoclinic crystal, there are 187 different elemental combinations and 6155 calculations herein
  Within the cubic crystal, there are 458 different elemental combinations and 9885 calculations herein
  Within the trigonal crystal, there are 100 different elemental combinations and 3592 calculations herein
  Within the tetragonal crystal, there are 315 different elemental combinations and 7670 calculations herein
  Within the triclinic crystal, there are 29 different elemental combinations and 749 calculations herein


Ok, so that method of organization was way too spammy. Let's try it a different way.

In [11]:
import pprint


print('There are %i total calculations.' % len(docs))

elements = {element for stoich in stoichs.values() for element in stoich}
print('There are %i total elements in our dataset.' % len(elements))

print('There are %i different crystal structures:' % len(structure_counts))
for crystal, count in structure_counts.items():
    print('    %s structures encompass %i of the calculations.' % (crystal, count))


print('There are %i different bulk structures:' % sum(chemical_varieties.values()))
for n_elements, n_bulks in sorted(chemical_varieties.items()):
    print('    %i different bulk structures with %i different components encompass %i of the calculations.'
          % (n_bulks, n_elements, ncalcs_by_nelements[n_elements]))

print('There are %i unique surfaces across all the structures.' % len(surfaces))

print('There are %i different sites across all the surfaces.' % len(sites))

print('There are %i different adsorbates:' % len(counts_by_ads))
for ads, count in counts_by_ads.items():
    print('    %s structures encompass %i of the calculations.' % (ads, count))

There are 47279 total calculations.
There are 52 total elements in our dataset.
There are 7 different crystal structures:
    hexagonal structures encompass 6889 of the calculations.
    orthorhombic structures encompass 12339 of the calculations.
    monoclinic structures encompass 6155 of the calculations.
    cubic structures encompass 9885 of the calculations.
    trigonal structures encompass 3592 of the calculations.
    tetragonal structures encompass 7670 of the calculations.
    triclinic structures encompass 749 of the calculations.
There are 1952 different bulk structures:
    61 different bulk structures with 1 different components encompass 5844 of the calculations.
    1057 different bulk structures with 2 different components encompass 31651 of the calculations.
    774 different bulk structures with 3 different components encompass 9139 of the calculations.
    54 different bulk structures with 4 different components encompass 636 of the calculations.
    6 different bu